In [1]:
import numpy as np
import pandas as pd
np.random.seed(7)

In [2]:
def toNumber(inputList):
    outList = []    
    for word in inputList:
        newWord = []
        for char in word:
            newWord.append(ord(char))
        outList.append(newWord)          
    return outList

def fitToSize(word, n):
    wordFit = word[:n]
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)    
    return wordFit

def fixSymbolEncode(x):
    
    if x == 8217:
        return 39
    elif x == 8220:
        return 34
    elif x == 8221:
        return 34
    elif x == 8230:
        return 46
    elif x == 8722:
        return 45
    elif x == 8211:
        return 45
    elif x == 8216:
        return 39
    elif x == 8226:
        return 46
    elif x == 8364:
        return 36
    return x

In [3]:
data = pd.read_csv('dates.csv')
Y = np.array(data.word)
X = np.array(data.label)

In [4]:
for i in range(len(X)):
    if type(X[i]) != str:
        X[i] = ''
for x in X:
    if type(x) != str:        
        print(x)

In [5]:
data = toNumber(X)

In [6]:
for i in range(len(data)):
    data[i] = fitToSize(data[i],8)

In [7]:
for i in range(len(data)):
    for j in range(len(data[i])):        
            data[i][j] = fixSymbolEncode(data[i][j])

In [8]:
maxl = 0;
maxa = []
for i in range(len(data)):
    for j in range(len(data[i])):        
           if data[i][j] > 126:
                maxl+=1                
print(maxl)

3042


In [9]:
labels = []
for y in Y:
    if y == 'NONE':
        labels.append(0)
    elif y == 'D_ST':
        labels.append(2)
    elif y == 'D_PT':
        labels.append(1)

In [10]:
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import Model

Using TensorFlow backend.


In [11]:
inputLayer = Input(dtype='int32', shape=(8,), name = 'inputLayer')
embeddingLayer = Embedding(input_dim = 9000, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer')(inputLayer)
lstmLayer = LSTM(1, return_sequences=False, consume_less='gpu', 
                                name = 'latmLayer')(embeddingLayer)

model = Model(input=inputLayer, output=lstmLayer)
model.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [12]:
data = np.array(data)
data.shape

(459952, 8)

In [69]:
data.shape

(459952, 8)

In [70]:
data[:10]

array([[ 82, 117, 115, 115, 105,  97,  39, 115],
       [ 76, 101, 110, 101, 110, 101, 114, 103],
       [105, 110,   0,   0,   0,   0,   0,   0],
       [ 99, 114, 105, 115, 105, 115,   0,   0],
       [100, 117, 101,   0,   0,   0,   0,   0],
       [116, 111,   0,   0,   0,   0,   0,   0],
       [109,  97, 110,  97, 103, 101, 109, 101],
       [105, 110, 101, 102, 102, 105,  99, 105],
       [ 77,  79,  83,  67,  79,  87,   0,   0],
       [ 77,  97, 114,   0,   0,   0,   0,   0]])

In [ ]:
model.fit(data, labels)

Epoch 1/10
50000/50000 [==============================] - 14s - loss: 0.0625 - acc: 0.9749     8  224/50000 [..............................] - ETA: 50s - loss: 0.1386 - acc: 0.9643 

In [15]:
answer = model.predict(data)

In [18]:
testdf = pd.DataFrame(answer, labels)
testdf.to_csv('testdf.csv')